In [1]:
from datasets import load_from_disk

alpaca_dataset = load_from_disk("data/alpaca")
print(alpaca_dataset)

# Show basic info and first few rows
for split, data in alpaca_dataset.items():
    print(f"\n--- {split.upper()} ---")
    print(data)
    print(data[0])
    break  # we just need one example for structure


DatasetDict({
    train: Dataset({
        features: ['output', 'input', 'instruction'],
        num_rows: 51760
    })
})

--- TRAIN ---
Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 51760
})
{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.', 'input

In [2]:
import pandas as pd

# Load the train split into a DataFrame
alpaca_df = alpaca_dataset["train"].to_pandas()

# Basic stats
print("Shape:", alpaca_df.shape)
print("\nColumn-wise missing values:")
print(alpaca_df.isna().sum())

# Count how many have empty 'input'
empty_input_count = (alpaca_df['input'].str.strip() == '').sum()
print(f"\nEmpty 'input' rows: {empty_input_count}/{len(alpaca_df)}")

# Preview a few examples where input is NOT empty
print("\nExamples with non-empty 'input':")
print(alpaca_df[alpaca_df['input'].str.strip() != ''].head(5).to_dict(orient='records'))


Shape: (51760, 3)

Column-wise missing values:
output         0
input          0
instruction    0
dtype: int64

Empty 'input' rows: 32603/51760

Examples with non-empty 'input':
[{'output': 'The fraction 4/16 is equivalent to 1/4 because both fractions represent the same value. A fraction can be simplified by dividing both the numerator and the denominator by a common factor. In this case, 4 is a common factor of both the numerator and the denominator of 4/16. When we divide both by 4, we get 4/4 = 1 and 16/4 = 4, so the simplified fraction is 1/4. Alternatively, we can think of this in terms of multiplication. For example, if we multiply the numerator and denominator of the fraction 1/4 by 4, we get (1x4)/(4x4), or 4/16. Since both fractions can be derived from the other through multiplication or division by the same number, they represent the same value and are equivalent.', 'input': '4/16', 'instruction': 'Explain why the following fraction is equivalent to 1/4'}, {'output': 'There 

In [3]:
import pandas as pd

alpaca_df = alpaca_dataset["train"].to_pandas()

# Merge instruction + input into one field
def merge_prompt(row):
    if row["input"].strip():
        return f"Instruction: {row['instruction'].strip()}\nInput: {row['input'].strip()}"
    else:
        return f"Instruction: {row['instruction'].strip()}"

alpaca_df["prompt"] = alpaca_df.apply(merge_prompt, axis=1)
alpaca_df["response"] = alpaca_df["output"].astype(str).str.strip()

# Keep only the useful columns
alpaca_clean = alpaca_df[["prompt", "response"]]

print(alpaca_clean.sample(3).to_dict(orient="records"))
print("\nFinal shape:", alpaca_clean.shape)


[{'prompt': 'Instruction: Explain what data preprocessing is and three steps used in the process.', 'response': 'Data preprocessing is an important step in the data analysis and machine learning process. It involves the transformation of raw data into a usable format, so that it can be effectively used in machine learning algorithms, predictive modeling or data mining techniques. The aim of data preprocessing is to improve the quality and accuracy of the data by eliminating any inconsistencies, errors or missing values, and to prepare the data for further analysis. \n\nThere are several steps involved in the data preprocessing process, some of which includes:\n\n1. Data Cleaning: This involves checking and correcting errors in the data such as duplicate records, mislabeled data points or invalid values. It also involves filling in missing values or removing records that do not meet certain criteria.\n\n2. Data Transformation: This involves converting the data to a standard format, so t

In [4]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # collapse multiple spaces/newlines
    text = text.strip()
    return text

alpaca_clean["prompt"] = alpaca_clean["prompt"].apply(clean_text)
alpaca_clean["response"] = alpaca_clean["response"].apply(clean_text)


/var/folders/85/5_4w11lj2vv3mvmb5v4wkqr40000gn/T/ipykernel_84458/3585068580.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alpaca_clean["prompt"] = alpaca_clean["prompt"].apply(clean_text)
/var/folders/85/5_4w11lj2vv3mvmb5v4wkqr40000gn/T/ipykernel_84458/3585068580.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alpaca_clean["response"] = alpaca_clean["response"].apply(clean_text)


In [5]:
import os
os.makedirs("data_cleaned", exist_ok=True)
alpaca_clean.to_json("data_cleaned/alpaca_clean.jsonl", orient="records", lines=True)
print("✅ Cleaned Alpaca dataset saved to data_cleaned/alpaca_clean.jsonl")


✅ Cleaned Alpaca dataset saved to data_cleaned/alpaca_clean.jsonl


In [6]:
import pandas as pd

# Load the OpenOrca subset
df_orca = pd.read_csv("data/open_orca_subset.csv")
print("Shape:", df_orca.shape)
print("\nColumn names:", df_orca.columns.tolist())

# Show the first few rows to understand structure
print("\nSample rows:")
print(df_orca.head(3).to_dict(orient="records"))


Shape: (100000, 4)

Column names: ['id', 'system_prompt', 'question', 'response']

Sample rows:
[{'id': 'niv.242684', 'system_prompt': nan, 'question': "You will be given a definition of a task first, then some input of the task.\nThis task is about using the specified sentence and converting the sentence to Resource Description Framework (RDF) triplets of the form (subject, predicate object). The RDF triplets generated must be such that the triplets accurately capture the structure and semantics of the input sentence. The input is a sentence and the output is a list of triplets of the form [subject, predicate, object] that capture the relationships present in the sentence. When a sentence has more than 1 RDF triplet possible, the output must contain all of them.\n\nAFC Ajax (amateurs)'s ground is Sportpark De Toekomst where Ajax Youth Academy also play.\nOutput:", 'response': '[\n  ["AFC Ajax (amateurs)", "has ground", "Sportpark De Toekomst"],\n  ["Ajax Youth Academy", "plays at", "S

In [7]:
import pandas as pd
df_orca = pd.read_csv("data/open_orca_subset.csv")

# Count system prompts
has_system = df_orca['system_prompt'].notna().sum()
print(f"Rows with system_prompt: {has_system}/{len(df_orca)}")

# Quick look at a few with system prompts
print("\nExamples WITH system_prompt:")
print(df_orca[df_orca['system_prompt'].notna()].head(3).to_dict(orient='records'))


Rows with system_prompt: 92427/100000

Examples WITH system_prompt:
[{'id': 'flan.564327', 'system_prompt': 'You are an AI assistant. You will be given a task. You must generate a detailed and long answer.', 'question': 'Generate an approximately fifteen-word sentence that describes all this data: Midsummer House eatType restaurant; Midsummer House food Chinese; Midsummer House priceRange moderate; Midsummer House customer rating 3 out of 5; Midsummer House near All Bar One', 'response': 'Midsummer House is a moderately priced Chinese restaurant with a 3/5 customer rating, located near All Bar One.'}, {'id': 'flan.1875913', 'system_prompt': 'You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.', 'question': 'What happens next in this paragraph?\n\nShe then rubs a needle on a cotton ball then pushing it onto a pencil and wrapping thread around it. She then holds up a box of a product and then pouring several liquids into a bowl. s

In [8]:
import pandas as pd
import numpy as np
import re
import os

df_orca = pd.read_csv("data/open_orca_subset.csv")

# Merge into single prompt field
def merge_prompt(row):
    sys_prompt = str(row["system_prompt"]) if not pd.isna(row["system_prompt"]) else ""
    question = str(row["question"]).strip()
    if sys_prompt.strip():
        return f"System: {sys_prompt.strip()}\n\nUser: {question}"
    else:
        return f"User: {question}"

df_orca["prompt"] = df_orca.apply(merge_prompt, axis=1)
df_orca["response"] = df_orca["response"].astype(str).str.strip()

# Light cleanup — remove weird spacing
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_orca["prompt"] = df_orca["prompt"].apply(clean_text)
df_orca["response"] = df_orca["response"].apply(clean_text)

# Keep only useful columns
df_orca_clean = df_orca[["prompt", "response"]]

# Quick preview
print(df_orca_clean.sample(3).to_dict(orient="records"))
print("\nFinal shape:", df_orca_clean.shape)


[{'prompt': 'System: You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer. User: Definition: In this task, you\'re given reviews from Amazon\'s food products and a summary of that review. Your task is to classify whether the given summary matches the original review. Generate "True" if the given review and its summary match, otherwise generate "False". Input: Reviews: Classic Lynch and a horrible transfer. Grainy in many scenes. My Dvd Edition looks better. Summary: The suspenders are fine, sort of...the packaging not so much. Output:', 'response': 'False'}, {'prompt': "System: You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old. User: Translate the following sentence to Czech: And I don't think there's a demon in this court Who could fill the cloven hooves", 'response': 'A já si nemyslím, že je tady v tomto soudu nějaký démon, který by dokázal zaplnit rozštěpené kopyto.

In [9]:
os.makedirs("data_cleaned", exist_ok=True)
df_orca_clean.to_json("data_cleaned/open_orca_clean.jsonl", orient="records", lines=True)
print("✅ Cleaned OpenOrca dataset saved to data_cleaned/open_orca_clean.jsonl")


✅ Cleaned OpenOrca dataset saved to data_cleaned/open_orca_clean.jsonl


In [1]:
import fasttext, json, random
from collections import Counter

model = fasttext.load_model("lid.176.bin")
print("✅ model loaded")

# sample and test
path = "data_cleaned/tamil_wikipedia_clean.jsonl"
sample_texts = []
with open(path, "r") as f:
    for i, line in enumerate(f):
        if i % 1000 == 0:
            sample_texts.append(json.loads(line)["text"])
        if len(sample_texts) >= 200:
            break

def safe_predict(text):
    labels, _ = model.predict(text[:500])
    return labels[0] if labels else "__label__unknown"

predictions = [safe_predict(t) for t in sample_texts]
from collections import Counter
counts = Counter(predictions)
print(counts)


✅ model loaded
Counter({'__label__ta': 177})


In [2]:
import json, fasttext, os
from tqdm import tqdm

model = fasttext.load_model("lid.176.bin")

input_path = "data_cleaned/tamil_wikipedia_clean.jsonl"
output_path = "data_cleaned/tamil_wikipedia_tamil_only.jsonl"

with open(input_path, "r") as infile, open(output_path, "w") as outfile:
    for line in tqdm(infile, desc="Filtering Tamil-only lines"):
        article = json.loads(line)
        text = article["text"]
        label, prob = model.predict(text[:500])
        if label[0] == "__label__ta" and prob[0] > 0.85:
            json.dump(article, outfile, ensure_ascii=False)
            outfile.write("\n")

print(f"✅ Tamil-only filtered corpus saved to: {output_path}")


Filtering Tamil-only lines: 176887it [00:09, 19049.01it/s]

✅ Tamil-only filtered corpus saved to: data_cleaned/tamil_wikipedia_tamil_only.jsonl


In [3]:
import json
import pandas as pd

file_path = "data/culturax_tamil_subset.jsonl"

# Read first 5 lines manually (to inspect structure)
with open(file_path, "r") as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        sample = json.loads(line)
        print(json.dumps(sample, indent=2, ensure_ascii=False))

# Also quickly check number of fields in first line
df_sample = pd.read_json(file_path, lines=True, nrows=50)
print("\nColumns:", df_sample.columns.tolist())
print("Shape:", df_sample.shape)


{
  "text": "குழந்தை பராமரிப்பு | தினகரன்\nHome குழந்தை பராமரிப்பு\nகுழந்தைகளின் செய்கை மொழியை எளிதில் புரிந்து கொள்ள இயலாது. அழுகையும், சிரிப்புமே குழந்தையின் அழகிய மொழிகள். அதனை உணர்ந்து குழந்தைகளை வளர்ப்பது என்பது பெற்றோருக்கே உண்டான தனிக்கலை. பச்சிளம் குழந்தைகளின் செயல்பாடுகள் புரியாமல் சில நேரங்களில் தாய்மார்கள் எரிச்சல் அடைவதும் உண்டு. இங்கு குழந்தைகளை எவ்வாறு கையாள வேண்டும்.\n1. குழந்தைகள் முதலில் விரும்புவது தாயின் அரவணைப்பையும், அருகாமையையும் தான். அந்த கதகதப்பு கிடைக்காத பட்சத்தில் அதற்காகவே அழுகையை தொடங்குகிறார்கள். எனவே குழந்தைகள் அழும் போது அவர்களை தூக்கி கொஞ்சினால் குழந்தைகள் உடனே அழுகையை நிறுத்திவிடுவார்கள். அந்த நேரத்தில் குழந்தைகளுக்கு உணவூட்டுதல், மசாஜ் அல்லது குளிக்க வைக்கவோ செய்யலாம்.\n2. குழந்தையைத் தூக்கும்போது முதுகுப் புறமாக அதிகமாகப் பிடித்துத் தூக்கக்கூடாது, அது குழந்தையின் தண்டுவடத்தைப் பாதிக்கும் என்று சிலர் சொல்வார்கள். ஆனால் உண்மையில்லை. பின்னால் பிடித்துத் தூக்குவது குழந்தையின் ரிப்ளெக்ஸ்' திறனை மேம்படுத்துகிறது. பின்புற மற்றும் கழுத்துத் தசைகளையும், தண்ட

In [4]:
import re
import pandas as pd

# Load a small chunk (e.g. 10k rows for fast inspection)
file_path = "data/culturax_tamil_subset.jsonl"
df = pd.read_json(file_path, lines=True, nrows=10000)

# Basic text stats
df["char_len"] = df["text"].str.len()
df["english_frac"] = df["text"].apply(lambda x: len(re.findall(r"[A-Za-z]", x)) / len(x) if len(x) > 0 else 0)

print("Basic stats:")
print(df[["char_len", "english_frac"]].describe())

# Threshold examples
high_english = df[df["english_frac"] > 0.3].sample(3, random_state=42)
short_texts = df[df["char_len"] < 100].sample(3, random_state=42)

print("\n--- High English Fraction Examples ---")
for t in high_english["text"]:
    print(t[:300], "\n")

print("\n--- Very Short Text Examples ---")
for t in short_texts["text"]:
    print(t[:300], "\n")
    

Basic stats:
            char_len  english_frac
count   10000.000000  10000.000000
mean     3455.811400      0.037460
std      5495.700495      0.055683
min       101.000000      0.000000
25%      1291.000000      0.006016
50%      2004.000000      0.018728
75%      3510.500000      0.044700
max    162315.000000      0.532642


ValueError: a must be greater than 0 unless no samples are taken

In [6]:
import re, json, unicodedata, os
from tqdm import tqdm

def normalize_tamil(text):
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"\s+", " ", text).strip()
    text = text.replace("–", "-").replace("“", '"').replace("”", '"')
    return text

cleaned = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    text = row["text"]
    english_frac = len(re.findall(r"[A-Za-z]", text)) / len(text)
    if english_frac < 0.25 and len(text) < 50000:
        record = row.to_dict()
        record["text"] = normalize_tamil(text)
        # 🧹 Convert pandas.Timestamp → str
        if "timestamp" in record and not isinstance(record["timestamp"], str):
            record["timestamp"] = str(record["timestamp"])
        cleaned.append(record)

# Save cleaned file
out_path = "data/processed/culturax_tamil_clean.jsonl"
os.makedirs(os.path.dirname(out_path), exist_ok=True)

with open(out_path, "w") as f:
    for item in cleaned:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"✅ Saved cleaned CulturaX Tamil subset: {len(cleaned)} rows → {out_path}")


100%|██████████| 10000/10000 [00:02<00:00, 4778.89it/s]


✅ Saved cleaned CulturaX Tamil subset: 9838 rows → data/processed/culturax_tamil_clean.jsonl


In [7]:
import pandas as pd, re

clean_path = "data/processed/culturax_tamil_clean.jsonl"

df_clean = pd.read_json(clean_path, lines=True, nrows=20)
print("✅ Loaded cleaned CulturaX Tamil subset (first 20 rows)")
print("Columns:", df_clean.columns.tolist())
print("Shape:", df_clean.shape)

# Sample 3 random rows
print("\n--- Sample Cleaned Entries ---\n")
for t in df_clean.sample(3, random_state=42)["text"]:
    print(t[:400], "\n---\n")

# Sanity check: English fraction (should be very low)
english_mean = df_clean["text"].apply(lambda x: len(re.findall(r"[A-Za-z]", x)) / len(x)).mean()
print(f"\nMean English fraction after cleaning: {english_mean:.4f}")


✅ Loaded cleaned CulturaX Tamil subset (first 20 rows)
Columns: ['text', 'timestamp', 'url', 'source', 'char_len', 'english_frac']
Shape: (20, 6)

--- Sample Cleaned Entries ---

குழந்தை பராமரிப்பு | தினகரன் Home குழந்தை பராமரிப்பு குழந்தைகளின் செய்கை மொழியை எளிதில் புரிந்து கொள்ள இயலாது. அழுகையும், சிரிப்புமே குழந்தையின் அழகிய மொழிகள். அதனை உணர்ந்து குழந்தைகளை வளர்ப்பது என்பது பெற்றோருக்கே உண்டான தனிக்கலை. பச்சிளம் குழந்தைகளின் செயல்பாடுகள் புரியாமல் சில நேரங்களில் தாய்மார்கள் எரிச்சல் அடைவதும் உண்டு. இங்கு குழந்தைகளை எவ்வாறு கையாள வேண்டும். 1. குழந்தைகள் முதலில் விரும்பு 
---

வடகொரியாவால் மறக்க முடியாத நான்கு அமெரிக்க வீரர்கள் - BBC News தமிழ் வடகொரியாவால் மறக்க முடியாத நான்கு அமெரிக்க வீரர்கள் சைமன் ஃபவுலர் பிபிசி செய்தியாளர் 29 டிசம்பர் 2017 https://www.bbc.com/tamil/global-42492727 படத்தின் காப்புரிமை North Korea Picture Library / Alamy Stock Photo Image caption வடகொரியாவால் மறக்க முடியாத நான்கு அமெரிக்க வீரர்கள் வட கொரியாவில் திரைப்படங்களில் அமெரிக்க நட்சத்திரங்கள் 
---

கிரேக்க

In [8]:
from datasets import load_from_disk

tamil_alpaca_path = "data/tamil_alpaca"
tamil_alpaca = load_from_disk(tamil_alpaca_path)

print(tamil_alpaca)
print("\n--- SAMPLE ---")
print(tamil_alpaca["train"][0])


DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text', 'system_prompt'],
        num_rows: 51876
    })
})

--- SAMPLE ---
{'instruction': 'ஆரோக்கியமாக இருப்பதற்கு மூன்று குறிப்புகளைக் கொடுங்கள்.', 'input': '', 'output': '1. சமச்சீரான மற்றும் சத்தான உணவை உண்ணுங்கள்: உங்கள் உணவில் பலவகையான பழங்கள் மற்றும் காய்கறிகள், மெலிந்த புரதம், முழு தானியங்கள் மற்றும் ஆரோக்கியமான கொழுப்புகள் உள்ளன என்பதை உறுதிப்படுத்திக் கொள்ளுங்கள். இது உங்கள் உடலைச் சிறப்பாகச் செயல்பட தேவையான ஊட்டச்சத்துக்களுடன் வழங்க உதவுகிறது மற்றும் நாள்பட்ட நோய்களைத் தடுக்க உதவுகிறது.\n\n2. வழக்கமான உடல் செயல்பாடுகளில் ஈடுபடுங்கள்: வலுவான எலும்புகள், தசைகள் மற்றும் இருதய ஆரோக்கியத்தை பராமரிக்க உடற்பயிற்சி முக்கியமானது. ஒவ்வொரு வாரமும் குறைந்தபட்சம் 150 நிமிட மிதமான ஏரோபிக் உடற்பயிற்சி அல்லது 75 நிமிட தீவிரமான உடற்பயிற்சியை இலக்காகக் கொள்ளுங்கள்.\n\n3. போதுமான தூக்கத்தைப் பெறுங்கள்: போதுமான தரமான தூக்கம் உடல் மற்றும் மன நலனுக்கு முக்கியமானது. இது மனநிலையை சீராக்க உதவுகிறது, அறிவாற்றல் ச

In [9]:
import random

# Sample a few random rows for manual inspection
for i in random.sample(range(len(tamil_alpaca["train"])), 3):
    row = tamil_alpaca["train"][i]
    print(f"\n--- SAMPLE {i} ---")
    print("Instruction:", row["instruction"][:200])
    if row["input"]:
        print("Input:", row["input"][:200])
    print("Output:", row["output"][:300])
    print("System Prompt:", row["system_prompt"][:150])



--- SAMPLE 18406 ---
Instruction: ஐந்து செயல்பாடுகளின் பட்டியலை உருவாக்கவும், இது ஒரு நல்ல வார இறுதி பயணத்தை உருவாக்கும்.
Output: 1. இயற்கையில் பயணம் செய்து உள்ளூர் பாதைகள் மற்றும் இயற்கைக்காட்சிகளை ஆராயுங்கள்.
2. இப்பகுதியின் வரலாறு மற்றும் கலாச்சாரத்தைப் பற்றி அறிய ஒரு வரலாற்றுச் சுற்றுலா செல்லவும் அல்லது அருங்காட்சியகத்தைப் பார்வையிடவும்.
3. சில புத்துணர்ச்சி மற்றும் சுய பாதுகாப்புக்காக ஸ்பா அல்லது சூடான நீரூற்றில் ஓய்வெடுக
System Prompt: ஒரு பணியை எவ்வாறு நிறைவேற்ற வேண்டும் என்று கூறும் அறிவுரை கீழே உள்ளது. வேண்டுகோளைப் பொருத்தமாக நிறைவு செய்கின்ற பதில் ஒன்றை எழுதுக.

--- SAMPLE 14832 ---
Instruction: உங்கள் சொந்த ஊரைப் பற்றி ஒரு லிமெரிக் எழுதுங்கள்.
Output: மிகவும் சிறிய மற்றும் மிகவும் பசுமையான ஒரு நகரத்தில்,
மிகவும் அழகான இடத்தை பார்க்க முடியவில்லை,
மக்கள் அனைவரும் புன்னகைக்கிறார்கள்,
மேலும் கூடுதல் மைல் செல்லவும்,
என் சொந்த ஊரில், வாழ்க்கை மிகவும் அமைதியாக இருக்கிறது.
System Prompt: ஒரு பணியை எவ்வாறு நிறைவேற்ற வேண்டும் என்று கூறும் அறிவுரை கீழே உள்ளது. வேண்டுகோளைப் பொருத்தமாக 

In [10]:
from datasets import load_from_disk
import json
from tqdm import tqdm

tamil_alpaca_path = "data/tamil_alpaca"
tamil_alpaca = load_from_disk(tamil_alpaca_path)

output_path = "data/processed/tamil_alpaca_clean.jsonl"

with open(output_path, "w") as f:
    for ex in tqdm(tamil_alpaca["train"]):
        # Build minimal schema
        clean_entry = {
            "instruction": ex["instruction"].strip(),
            "input": ex["input"].strip() if ex["input"] else "",
            "output": ex["output"].strip()
        }
        json.dump(clean_entry, f, ensure_ascii=False)
        f.write("\n")

print(f"✅ Saved Tamil Alpaca Clean JSONL → {output_path}")


100%|██████████| 51876/51876 [00:01<00:00, 36714.07it/s]

✅ Saved Tamil Alpaca Clean JSONL → data/processed/tamil_alpaca_clean.jsonl


In [14]:
from datasets import load_from_disk

# Load the dataset
nlpc_uom = load_from_disk("data/nlpc_uom_en_ta")

# Export first 20 samples for inspection
nlpc_uom["train"].select(range(20)).to_json("nlpc_sample.jsonl", force_ascii=False, orient="records", lines=True)

print("✅ Exported first 20 rows → nlpc_sample.jsonl")


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

✅ Exported first 20 rows → nlpc_sample.jsonl


NameError: name 'nlpc_uom_en_ta' is not defined